In [1]:
import pandas as pd
import numpy as np
seed = np.random.seed(42)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)

%matplotlib inline
best_model = None
best_score = 0
hospital_name = 'host3'

In [2]:
data = pd.read_csv('../data/hospital' + str(3) + '_data.csv')

In [3]:
from sklearn.preprocessing import Imputer,StandardScaler,LabelEncoder
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE

/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
data_e = data[data['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
y_df = data_e['AFclass'].copy()
x_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (274, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.005
[MICE] Starting imputation round 2/110, elapsed time 0.033
[MICE] Starting imputation round 3/110, elapsed time 0.056
[MICE] Starting imputation round 4/110, elapsed time 0.081
[MICE] Starting imputation round 5/110, elapsed time 0.104
[MICE] Starting imputation round 6/110, elapsed time 0.128
[MICE] Starting imputation round 7/110, elapsed time 0.151
[MICE] Starting imputation round 8/110, elapsed time 0.174
[MICE] Starting imputation round 9/110, elapsed time 0.197
[MICE] Starting imputation round 10/110, elapsed time 0.232
[MICE] Starting imputation round 11/110, elapsed time 0.260
[MICE] Starting imputation round 12/110, elapsed time 0.284
[MICE] Starting imputation round 13/110, elapsed time 0.308
[MICE] Starting imputation round 14/110, elapsed time 0.331
[MICE] Starting imputation round 15/110, elapsed time 0.354
[MICE] Starting imputation round 16/110, elapsed time 0.378
[MI

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

#f1_scorer = make_scorer(f1_score, average='macro',labels=Y.unique().tolist())
#myscorer = f1_scorer
myscorer = 'roc_auc'

In [6]:
balancer = SMOTE(random_state=42)

In [7]:
def show_f_importance(estimator,train_data):
    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(train_data.shape[1]):
        print("%d. feature %d - %s (%f)" % (f + 1, indices[f],train_data.columns[indices[f]], importances[indices[f]]))


In [8]:
param_grid = {"clf__strategy": ['most_frequent','stratified','prior','uniform']}

dummy = DummyClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', dummy))
pipeline = Pipeline(plist)

gs_dummy = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [9]:
gs_dummy.fit(x_df, y_df);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.2s finished


In [10]:
gs_dummy.best_params_

{'clf__strategy': 'stratified'}

In [11]:
gs_dummy.best_score_

0.5046998479802258

In [12]:
param_grid = {"clf__max_depth": [10,30,50],
              "clf__criterion": ['gini', 'entropy'],
              "clf__n_estimators": [100,200,500,1000],
              "clf__random_state": [42],
              "clf__class_weight" : [None, 'balanced']}
# param_grid = {"classifier__max_depth": [30],
#               "classifier__n_estimators": [1000]}


rf = RandomForestClassifier()

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', rf))
pipeline = Pipeline(plist)

gs_rf = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [13]:
gs_rf.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  2.3min finished


In [14]:
gs_rf.best_params_

{'clf__class_weight': None,
 'clf__criterion': 'entropy',
 'clf__max_depth': 30,
 'clf__n_estimators': 200,
 'clf__random_state': 42}

In [15]:
gs_rf.best_score_


0.5301850289873158

In [16]:
if gs_rf.best_score_ > best_score:
    best_model = gs_rf
    best_score = gs_rf.best_score_

In [17]:
show_f_importance(gs_rf.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 66 - patsex (0.032525)
2. feature 48 - Parea_ECG_aVL (0.028793)
3. feature 4 - QRS (0.027798)
4. feature 62 - Pfwhm_ECG_V9 (0.026390)
5. feature 3 - QTn (0.024675)
6. feature 5 - PQ (0.023805)
7. feature 55 - Pfwhm_ECG_V3 (0.023344)
8. feature 39 - Parea_ECG_V3 (0.022000)
9. feature 50 - Pfwhm_ECG_I (0.021586)
10. feature 57 - Pfwhm_ECG_V4 (0.020048)
11. feature 36 - Parea_ECG_III (0.020038)
12. feature 40 - Parea_ECG_V3R (0.019083)
13. feature 34 - Parea_ECG_I (0.018291)
14. feature 56 - Pfwhm_ECG_V3R (0.018213)
15. feature 41 - Parea_ECG_V4 (0.018015)
16. feature 44 - Parea_ECG_V6 (0.017827)
17. feature 10 - PCarea (0.017625)
18. feature 54 - Pfwhm_ECG_V2 (0.017600)
19. feature 63 - Pfwhm_ECG_aVF (0.017132)
20. feature 16 - Pdur16std (0.017085)
21. feature 6 - PCpos (0.017073)
22. feature 38 - Parea_ECG_V2 (0.016587)
23. feature 43 - Parea_ECG_V5 (0.016273)
24. feature 1 - QTm_old (0.016241)
25. feature 17 - Pdur16iqr (0.015792)
26. feature 45 - Parea_ECG_

In [18]:
param_grid = {"clf__max_depth": [3,5,8],
              "clf__n_estimators": [100,200,500,1000],
              "clf__min_samples_leaf": [1,3,5,10]}
# param_grid = {"classifier__n_estimators": [50],
#               "classifier__max_depth": [5],
# 

gb = GradientBoostingClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', gb))
pipeline = Pipeline(plist)
gs_gb = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [19]:
gs_gb.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.3min finished


In [20]:
gs_gb.best_params_

{'clf__max_depth': 8, 'clf__min_samples_leaf': 1, 'clf__n_estimators': 100}

In [21]:
gs_gb.best_score_

0.5321862392550061

In [22]:
if gs_gb.best_score_ > best_score:
    best_model = gs_gb
    best_score = gs_gb.best_score_

In [23]:
show_f_importance(gs_gb.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 48 - Parea_ECG_aVL (0.072476)
2. feature 66 - patsex (0.053093)
3. feature 55 - Pfwhm_ECG_V3 (0.042382)
4. feature 64 - Pfwhm_ECG_aVL (0.035687)
5. feature 54 - Pfwhm_ECG_V2 (0.034804)
6. feature 4 - QRS (0.030267)
7. feature 17 - Pdur16iqr (0.027897)
8. feature 16 - Pdur16std (0.027349)
9. feature 3 - QTn (0.027027)
10. feature 50 - Pfwhm_ECG_I (0.022879)
11. feature 67 - Age (0.021286)
12. feature 39 - Parea_ECG_V3 (0.018795)
13. feature 38 - Parea_ECG_V2 (0.017829)
14. feature 57 - Pfwhm_ECG_V4 (0.017121)
15. feature 63 - Pfwhm_ECG_aVF (0.016730)
16. feature 25 - Pdur_ECG_V4 (0.016650)
17. feature 47 - Parea_ECG_aVF (0.016050)
18. feature 0 - RR (0.015743)
19. feature 21 - Pdur_ECG_V1 (0.014747)
20. feature 37 - Parea_ECG_V1 (0.014234)
21. feature 53 - Pfwhm_ECG_V1 (0.013998)
22. feature 42 - Parea_ECG_V4R (0.013938)
23. feature 5 - PQ (0.013177)
24. feature 46 - Parea_ECG_V9 (0.013035)
25. feature 2 - QR (0.012984)
26. feature 44 - Parea_ECG_V6 (0.012414

In [24]:
param_grid = {"clf__learning_rate": [0.1,0.3,0.7,0.8],
              "clf__n_estimators": [100,1000,2000,2500,3000,5000]}

ab = AdaBoostClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', ab))
pipeline = Pipeline(plist)

gs_ab = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [25]:
gs_ab.fit(x_df, y_df);

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  6.4min finished


In [26]:
gs_ab.best_params_

{'clf__learning_rate': 0.1, 'clf__n_estimators': 100}

In [27]:
gs_ab.best_score_

0.555679473285855

In [28]:

if gs_ab.best_score_ > best_score:
    best_model = gs_ab
    best_score = gs_ab.best_score_

In [29]:
show_f_importance(gs_ab.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 66 - patsex (0.110000)
2. feature 5 - PQ (0.100000)
3. feature 55 - Pfwhm_ECG_V3 (0.080000)
4. feature 4 - QRS (0.080000)
5. feature 50 - Pfwhm_ECG_I (0.070000)
6. feature 62 - Pfwhm_ECG_V9 (0.070000)
7. feature 63 - Pfwhm_ECG_aVF (0.060000)
8. feature 48 - Parea_ECG_aVL (0.050000)
9. feature 44 - Parea_ECG_V6 (0.050000)
10. feature 40 - Parea_ECG_V3R (0.050000)
11. feature 3 - QTn (0.050000)
12. feature 37 - Parea_ECG_V1 (0.040000)
13. feature 54 - Pfwhm_ECG_V2 (0.030000)
14. feature 38 - Parea_ECG_V2 (0.030000)
15. feature 16 - Pdur16std (0.030000)
16. feature 6 - PCpos (0.030000)
17. feature 34 - Parea_ECG_I (0.020000)
18. feature 18 - Pdur_ECG_I (0.010000)
19. feature 43 - Parea_ECG_V5 (0.010000)
20. feature 45 - Parea_ECG_V8 (0.010000)
21. feature 1 - QTm_old (0.010000)
22. feature 36 - Parea_ECG_III (0.010000)
23. feature 67 - Age (0.000000)
24. feature 19 - Pdur_ECG_II (0.000000)
25. feature 21 - Pdur_ECG_V1 (0.000000)
26. feature 22 - Pdur_ECG_V2 (0.

In [30]:
param_grid = {"clf__max_depth": [3,5,8,10,15,30],
              "clf__n_estimators": [100,200,500,1000,1500],
              "clf__min_samples_leaf": [1,3,5,10]}

et = ExtraTreesClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', et))
pipeline = Pipeline(plist)

gs_et = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [31]:
gs_et.fit(x_df, y_df);

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  4.1min finished


In [32]:
gs_et.best_params_

{'clf__max_depth': 3, 'clf__min_samples_leaf': 10, 'clf__n_estimators': 200}

In [33]:
gs_et.best_score_

0.5831128636036998

In [34]:
if gs_et.best_score_ > best_score:
    best_model = gs_et
    best_score = gs_et.best_score_

In [35]:
show_f_importance(gs_et.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 66 - patsex (0.154331)
2. feature 48 - Parea_ECG_aVL (0.055500)
3. feature 4 - QRS (0.032543)
4. feature 5 - PQ (0.030863)
5. feature 50 - Pfwhm_ECG_I (0.028651)
6. feature 34 - Parea_ECG_I (0.027617)
7. feature 56 - Pfwhm_ECG_V3R (0.025312)
8. feature 6 - PCpos (0.022897)
9. feature 62 - Pfwhm_ECG_V9 (0.022698)
10. feature 57 - Pfwhm_ECG_V4 (0.021910)
11. feature 45 - Parea_ECG_V8 (0.020363)
12. feature 41 - Parea_ECG_V4 (0.019226)
13. feature 55 - Pfwhm_ECG_V3 (0.018942)
14. feature 39 - Parea_ECG_V3 (0.018583)
15. feature 46 - Parea_ECG_V9 (0.017015)
16. feature 38 - Parea_ECG_V2 (0.016769)
17. feature 16 - Pdur16std (0.016675)
18. feature 3 - QTn (0.016306)
19. feature 63 - Pfwhm_ECG_aVF (0.014977)
20. feature 17 - Pdur16iqr (0.014626)
21. feature 11 - P_LoffEon (0.014469)
22. feature 26 - Pdur_ECG_V4R (0.013659)
23. feature 31 - Pdur_ECG_aVF (0.013528)
24. feature 42 - Parea_ECG_V4R (0.012920)
25. feature 40 - Parea_ECG_V3R (0.012894)
26. feature 44 - P

In [36]:
cdata = pd.read_csv('../data/central' +'_data.csv')
cdata.shape

(275, 73)

In [37]:
data_e = cdata[cdata['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
cy_df = data_e['AFclass'].copy()
cx_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (270, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.002
[MICE] Starting imputation round 2/110, elapsed time 0.067
[MICE] Starting imputation round 3/110, elapsed time 0.103
[MICE] Starting imputation round 4/110, elapsed time 0.147
[MICE] Starting imputation round 5/110, elapsed time 0.173
[MICE] Starting imputation round 6/110, elapsed time 0.221
[MICE] Starting imputation round 7/110, elapsed time 0.272
[MICE] Starting imputation round 8/110, elapsed time 0.296
[MICE] Starting imputation round 9/110, elapsed time 0.323
[MICE] Starting imputation round 10/110, elapsed time 0.383
[MICE] Starting imputation round 11/110, elapsed time 0.419
[MICE] Starting imputation round 12/110, elapsed time 0.484
[MICE] Starting imputation round 13/110, elapsed time 0.520
[MICE] Starting imputation round 14/110, elapsed time 0.583
[MICE] Starting imputation round 15/110, elapsed time 0.608
[MICE] Starting imputation round 16/110, elapsed time 0.632
[MI

In [40]:
from sklearn.metrics import roc_auc_score
p = best_model.predict(cx_df);
scores = pd.DataFrame({hospital_name: p})
scores.to_csv('../data/' + hospital_name +'_predictions.csv', index=False)
np.save('../data/' + hospital_name +'_acc',best_score)
roc_auc_score(cy_df, p)

0.47211230450348596

In [41]:
best_score

0.5831128636036998

In [39]:
from sklearn.externals import joblib
joblib.dump(best_model.best_estimator_, '../data/' + hospital_name + '_model.pkl')

['../data/host3_model.pkl']